In [1]:
import numpy as np
import pickle
from collections import defaultdict
import random


In [2]:
class TicTacToe:
    def __init__(self):
        self.board = np.zeros((3, 3), dtype=int)
        self.current_player = 1  # 1 for X, -1 for O
        
    def reset(self):
        """Reset the game board"""
        self.board = np.zeros((3, 3), dtype=int)
        self.current_player = 1
        return self.get_state()
    
    def get_state(self):
        """Return current board state as a tuple (for hashing)"""
        return tuple(self.board.flatten())
    
    def available_actions(self):
        """Return list of available positions"""
        return [(i, j) for i in range(3) for j in range(3) if self.board[i][j] == 0]
    
    def make_move(self, position):
        """Make a move and return (next_state, reward, done)"""
        if self.board[position] != 0:
            return self.get_state(), -10, True  # Invalid move penalty
        
        self.board[position] = self.current_player
        
        # Check for win
        if self.check_winner(self.current_player):
            return self.get_state(), 1, True  # Win
        
        # Check for draw
        if len(self.available_actions()) == 0:
            return self.get_state(), 0.5, True  # Draw
        
        # Switch player
        self.current_player *= -1
        return self.get_state(), 0, False  # Continue game
    
    def check_winner(self, player):
        """Check if the given player has won"""
        # Check rows
        for i in range(3):
            if np.all(self.board[i, :] == player):
                return True
        
        # Check columns
        for j in range(3):
            if np.all(self.board[:, j] == player):
                return True
        
        # Check diagonals
        if np.all(np.diag(self.board) == player):
            return True
        if np.all(np.diag(np.fliplr(self.board)) == player):
            return True
        
        return False
    
    def display(self):
        """Display the board"""
        symbols = {0: '.', 1: 'X', -1: 'O'}
        for row in self.board:
            print(' '.join([symbols[cell] for cell in row]))
        print()


In [3]:
class QLearningAgent:
    def __init__(self, player_id=1, alpha=0.1, gamma=0.9, epsilon=0.2):
        """
        Q-Learning Agent for Tic-Tac-Toe
        
        Parameters:
        - player_id: 1 for X, -1 for O
        - alpha: learning rate
        - gamma: discount factor
        - epsilon: exploration rate for epsilon-greedy policy
        """
        self.player_id = player_id
        self.alpha = alpha  # Learning rate
        self.gamma = gamma  # Discount factor
        self.epsilon = epsilon  # Exploration rate
        self.q_table = defaultdict(float)  # Q-value table
        self.state_history = []  # Store states during episode
        
    def get_q_value(self, state, action):
        """Get Q-value for state-action pair"""
        return self.q_table[(state, action)]
    
    def choose_action(self, game, training=True):
        """Choose action using epsilon-greedy policy"""
        available = game.available_actions()
        
        if not available:
            return None
        
        state = game.get_state()
        
        # Epsilon-greedy: explore vs exploit
        if training and random.random() < self.epsilon:
            return random.choice(available)  # Explore
        
        # Exploit: choose action with highest Q-value
        q_values = [self.get_q_value(state, action) for action in available]
        max_q = max(q_values)
        
        # If multiple actions have same max Q-value, choose randomly among them
        best_actions = [action for action, q in zip(available, q_values) if q == max_q]
        return random.choice(best_actions)
    
    def update_q_value(self, state, action, reward, next_state, next_available_actions):
        """Update Q-value using Q-learning formula"""
        current_q = self.get_q_value(state, action)
        
        # Calculate max Q-value for next state
        if next_available_actions:
            max_next_q = max([self.get_q_value(next_state, a) for a in next_available_actions])
        else:
            max_next_q = 0
        
        # Q-learning update rule: Q(s,a) = Q(s,a) + α[r + γ*max(Q(s',a')) - Q(s,a)]
        new_q = current_q + self.alpha * (reward + self.gamma * max_next_q - current_q)
        self.q_table[(state, action)] = new_q
    
    def save_model(self, filename):
        """Save Q-table to file"""
        with open(filename, 'wb') as f:
            pickle.dump(dict(self.q_table), f)
    
    def load_model(self, filename):
        """Load Q-table from file"""
        with open(filename, 'rb') as f:
            self.q_table = defaultdict(float, pickle.load(f))


In [4]:
def train_agents(episodes=50000, verbose=True):
    """
    Train two Q-learning agents by playing against each other
    """
    game = TicTacToe()
    agent1 = QLearningAgent(player_id=1, alpha=0.1, gamma=0.9, epsilon=0.2)
    agent2 = QLearningAgent(player_id=-1, alpha=0.1, gamma=0.9, epsilon=0.2)
    
    wins = {1: 0, -1: 0, 0: 0}  # Track wins for X, O, and draws
    
    for episode in range(episodes):
        state = game.reset()
        episode_history = []  # Store (agent, state, action) for this episode
        
        done = False
        while not done:
            current_agent = agent1 if game.current_player == 1 else agent2
            
            # Choose and make action
            action = current_agent.choose_action(game, training=True)
            if action is None:
                break
            
            prev_state = game.get_state()
            next_state, reward, done = game.make_move(action)
            
            # Store for later update
            episode_history.append((current_agent, prev_state, action, reward, next_state, done))
        
        # Determine winner
        if game.check_winner(1):
            winner = 1
            wins[1] += 1
        elif game.check_winner(-1):
            winner = -1
            wins[-1] += 1
        else:
            winner = 0
            wins[0] += 1
        
        # Update Q-values for both agents
        for i, (agent, state, action, reward, next_state, done) in enumerate(episode_history):
            # Adjust reward based on outcome
            if done:
                if game.check_winner(agent.player_id):
                    final_reward = 1  # Win
                elif game.check_winner(-agent.player_id):
                    final_reward = -1  # Loss
                else:
                    final_reward = 0.5  # Draw
            else:
                final_reward = 0  # Intermediate move
            
            next_available = game.available_actions() if not done else []
            agent.update_q_value(state, action, final_reward, next_state, next_available)
        
        # Print progress
        if verbose and (episode + 1) % 10000 == 0:
            total = episode + 1
            print(f"Episode {episode + 1}/{episodes}")
            print(f"  X wins: {wins[1]} ({wins[1]/total*100:.1f}%)")
            print(f"  O wins: {wins[-1]} ({wins[-1]/total*100:.1f}%)")
            print(f"  Draws: {wins[0]} ({wins[0]/total*100:.1f}%)")
            print(f"  Q-table size: {len(agent1.q_table)}")
            print()
    
    return agent1, agent2, wins

# Train the agents
print("Training Q-Learning agents...\n")
trained_agent1, trained_agent2, training_stats = train_agents(episodes=50000)
print("\nTraining completed!")


Training Q-Learning agents...

Episode 10000/50000
  X wins: 6379 (63.8%)
  O wins: 2852 (28.5%)
  Draws: 769 (7.7%)
  Q-table size: 13751

Episode 20000/50000
  X wins: 13027 (65.1%)
  O wins: 5619 (28.1%)
  Draws: 1354 (6.8%)
  Q-table size: 14371

Episode 30000/50000
  X wins: 19670 (65.6%)
  O wins: 8434 (28.1%)
  Draws: 1896 (6.3%)
  Q-table size: 14548

Episode 40000/50000
  X wins: 26399 (66.0%)
  O wins: 11205 (28.0%)
  Draws: 2396 (6.0%)
  Q-table size: 14626

Episode 50000/50000
  X wins: 33077 (66.2%)
  O wins: 13983 (28.0%)
  Draws: 2940 (5.9%)
  Q-table size: 14670


Training completed!


In [5]:
def test_against_random(agent, num_games=100, agent_player=1):
    """
    Test trained agent against random player
    """
    game = TicTacToe()
    wins = {1: 0, -1: 0, 0: 0}
    
    for _ in range(num_games):
        state = game.reset()
        done = False
        
        while not done:
            if game.current_player == agent_player:
                # Trained agent's turn
                action = agent.choose_action(game, training=False)
            else:
                # Random player's turn
                available = game.available_actions()
                action = random.choice(available) if available else None
            
            if action is None:
                break
            
            next_state, reward, done = game.make_move(action)
        
        # Record result
        if game.check_winner(1):
            wins[1] += 1
        elif game.check_winner(-1):
            wins[-1] += 1
        else:
            wins[0] += 1
    
    print(f"\nTest Results (Agent as {'X' if agent_player == 1 else 'O'} vs Random Player):")
    print(f"  X wins: {wins[1]} ({wins[1]/num_games*100:.1f}%)")
    print(f"  O wins: {wins[-1]} ({wins[-1]/num_games*100:.1f}%)")
    print(f"  Draws: {wins[0]} ({wins[0]/num_games*100:.1f}%)")
    
    return wins

# Test the trained agent
test_results = test_against_random(trained_agent1, num_games=1000, agent_player=1)



Test Results (Agent as X vs Random Player):
  X wins: 833 (83.3%)
  O wins: 110 (11.0%)
  Draws: 57 (5.7%)


In [6]:
def play_against_human(agent, agent_player=1):
    """
    Play Tic-Tac-Toe against the trained agent
    """
    game = TicTacToe()
    state = game.reset()
    
    print("\nTic-Tac-Toe: You vs AI")
    print(f"You are {'O' if agent_player == 1 else 'X'}")
    print(f"AI is {'X' if agent_player == 1 else 'O'}")
    print("\nEnter moves as 'row,col' (0-2), e.g., '0,1' for top-middle\n")
    
    game.display()
    done = False
    
    while not done:
        if game.current_player == agent_player:
            # AI's turn
            print("AI is thinking...")
            action = agent.choose_action(game, training=False)
            print(f"AI plays: {action}")
        else:
            # Human's turn
            while True:
                try:
                    move = input("Your move (row,col): ")
                    row, col = map(int, move.split(','))
                    if (row, col) in game.available_actions():
                        action = (row, col)
                        break
                    else:
                        print("Invalid move! That position is taken.")
                except:
                    print("Invalid input! Use format: row,col (e.g., 1,2)")
        
        next_state, reward, done = game.make_move(action)
        game.display()
    
    # Announce winner
    if game.check_winner(1):
        winner = "X"
    elif game.check_winner(-1):
        winner = "O"
    else:
        winner = None
    
    if winner:
        if (winner == 'X' and agent_player == 1) or (winner == 'O' and agent_player == -1):
            print("🤖 AI wins!")
        else:
            print("🎉 You win!")
    else:
        print("🤝 It's a draw!")

# Play a game (uncomment to play)
# play_against_human(trained_agent1, agent_player=1)


In [7]:
def evaluate_agent_strength(agent, num_games=100):
    """
    Evaluate agent by playing against random player from both positions
    """
    print("\n" + "="*50)
    print("AGENT EVALUATION")
    print("="*50)
    
    # Test as X (first player)
    print("\n1. Agent playing as X (first):")
    results_x = test_against_random(agent, num_games=num_games, agent_player=1)
    
    # Test as O (second player)
    print("\n2. Agent playing as O (second):")
    results_o = test_against_random(agent, num_games=num_games, agent_player=-1)
    
    print("\n" + "="*50)
    print(f"Q-table contains {len(agent.q_table)} state-action pairs")
    print("="*50)

# Evaluate the trained agent
evaluate_agent_strength(trained_agent1, num_games=1000)



AGENT EVALUATION

1. Agent playing as X (first):

Test Results (Agent as X vs Random Player):
  X wins: 819 (81.9%)
  O wins: 118 (11.8%)
  Draws: 63 (6.3%)

2. Agent playing as O (second):

Test Results (Agent as O vs Random Player):
  X wins: 575 (57.5%)
  O wins: 298 (29.8%)
  Draws: 127 (12.7%)

Q-table contains 15453 state-action pairs


## Approach 2

In [8]:
# Simple Tic-Tac-Toe with Q-learning
# This code is intentionally compact and easy to memorize.
# It trains an agent (X) to play against a random opponent (O).
# Run in one cell (e.g. Google Colab). No external libs except numpy.

import random
import numpy as np
from collections import defaultdict

class TicTacToe:
    def __init__(self):
        self.reset()

    def reset(self):
        self.board = [' ']*9
        self.done = False
        self.winner = None
        return self._state()

    def _state(self):
        return ''.join(self.board)

    def available_actions(self):
        return [i for i, v in enumerate(self.board) if v == ' ']

    def step(self, action, mark):
        if self.board[action] != ' ':
            raise ValueError("Invalid action")
        self.board[action] = mark
        self._check_game()
        return self._state(), self._reward(mark), self.done

    def _check_game(self):
        b = self.board
        lines = [(0,1,2),(3,4,5),(6,7,8),(0,3,6),(1,4,7),(2,5,8),(0,4,8),(2,4,6)]
        for a,b1,c in lines:
            if self.board[a]==self.board[b1]==self.board[c] != ' ':
                self.done = True
                self.winner = self.board[a]
                return
        if ' ' not in self.board:
            self.done = True
            self.winner = None

    def _reward(self, last_mark):
        if not self.done:
            return 0
        if self.winner is None:
            return 0.5  # small reward for draw
        return 1 if self.winner == last_mark else -1

# Q-learning agent
class QAgent:
    def __init__(self, lr=0.5, gamma=0.9, eps=0.2):
        self.q = defaultdict(lambda: np.zeros(9))
        self.lr = lr; self.gamma=gamma; self.eps=eps

    def choose(self, state, available):
        if random.random() < self.eps:
            return random.choice(available)
        qvals = self.q[state].copy()
        # mask unavailable actions very low
        for a in range(9):
            if a not in available:
                qvals[a] = -999
        return int(np.argmax(qvals))

    def learn(self, s, a, r, s2, available2):
        qsa = self.q[s][a]
        if s2 is None:
            target = r
        else:
            # value for next state (only consider available actions)
            next_q = self.q[s2].copy()
            for idx in range(9):
                if idx not in available2:
                    next_q[idx] = -999
            target = r + self.gamma * np.max(next_q)
        self.q[s][a] += self.lr * (target - qsa)

# Training: agent plays as 'X' and learns, opponent plays random 'O'
def train(episodes=5000):
    env = TicTacToe()
    agent = QAgent(lr=0.5, gamma=0.9, eps=0.3)
    for ep in range(episodes):
        state = env.reset()
        player = 'X' if random.random() < 0.5 else 'O'
        last = None  # store (s,a,mark)
        while True:
            if player == 'X':
                avail = env.available_actions()
                a = agent.choose(state, avail)
                s2, r, done = env.step(a, 'X')
                avail2 = env.available_actions() if not done else []
                # learn from X's move
                agent.learn(state, a, r, s2 if not done else None, avail2)
                state = s2
                if done: break
                player = 'O'
            else:
                # random opponent
                avail = env.available_actions()
                a = random.choice(avail)
                s2, r, done = env.step(a, 'O')
                # penalize agent if O wins on this move
                if done:
                    # if O won, give negative reward to previous X move (handled above when X moved)
                    pass
                state = s2
                if done: break
                player = 'X'
        # slowly decay epsilon
        if agent.eps > 0.05:
            agent.eps *= 0.9995
    return agent

def evaluate(agent, games=1000):
    env = TicTacToe()
    wins = draws = losses = 0
    for _ in range(games):
        state = env.reset()
        player = 'X' if random.random() < 0.5 else 'O'
        while True:
            if player == 'X':
                a = agent.choose(state, env.available_actions())
                state, r, done = env.step(a, 'X')
                if done:
                    if env.winner == 'X': wins += 1
                    elif env.winner is None: draws += 1
                    else: losses += 1
                    break
                player = 'O'
            else:
                a = random.choice(env.available_actions())
                state, r, done = env.step(a, 'O')
                if done:
                    if env.winner == 'X': wins += 1
                    elif env.winner is None: draws += 1
                    else: losses += 1
                    break
                player = 'X'
    return wins, draws, losses

# Run training and evaluation
agent = train(episodes=8000)
w,d,l = evaluate(agent, games=1000)
print(f"Out of 1000 games vs random: Wins={w}, Draws={d}, Losses={l}")


Out of 1000 games vs random: Wins=660, Draws=40, Losses=300
